In [1]:
%matplotlib notebook

import os
import sys
from operator import itemgetter
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from ipyleaflet import (Map, GeoJSON)

sns.set()

from sklearn.cluster import DBSCAN

import matplotlib.dates as mdates
from matplotlib.colors import rgb2hex
import json

def get_geojson(features):
    return {
        'type': 'FeatureCollection',
        'features': features
    }

def save_geojson(features, directory, file_name):
    if not os.path.exists(directory): os.makedirs(directory)
    f = os.path.join(directory, file_name + '.geojson')
    geojson = {
      'type': 'FeatureCollection',
      'features': features
    }
    with open(f, 'w') as outfile:
        json.dump(geojson, outfile, indent = 4)
    print('Saved to ' + f)

def to_geojson(df, groupby, lat, lng, cols, dumps=True):

    def get_features(row, color):
        properties = { k: str(v) for k,v in zip(cols,[row[col] for col in cols]) }
        properties['marker-color'] = rgb2hex(color[:3])
        return {
            'type': 'Feature',
                'geometry': {
                'type': 'Point',
                'coordinates': [row[lng], row[lat]]
            },
            'properties': properties
        }

    clusters = df.groupby(groupby)

    features = []
    colors = plt.cm.Spectral(np.linspace(0, 1, len(clusters)))
    for name, group in clusters:
        i = np.random.randint(colors.shape[0])
        color = colors[i]
        group.apply(lambda row: features.append(get_features(row, color)), axis=1)
        colors = np.delete(colors, i, 0)

    if dumps:
        return json.dumps(get_geojson(features))
    return get_geojson(features)

In [2]:
nb_dir = os.path.normpath(os.path.join(os.getcwd(), '..'))
os.listdir(nb_dir)
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

f = '../data/UTSEUS-shanghai-dianping.db'

In [25]:
import sqlite3
conn = sqlite3.connect(f)
cursor = conn.cursor()
cursor.execute("""
SELECT tag, longitude, latitude, c.category, avg_rating, avg_price, name, city
FROM venues v
JOIN venue_categories vc ON (v.business_id = vc.business_id)
    JOIN categories c ON (vc.category = c.category)
        JOIN venues_tags vt ON (v.business_id = vt.business_id)
WHERE
 (c.parent = '美食' OR c.category = '美食')
    AND longitude > 115
    AND longitude < 123
    AND avg_price > 0
""")
venues = pd.DataFrame(cursor.fetchall())
venues.columns = ['cn_tag', 'longitude', 'latitude', 'category', 'avg_rating', 'avg_price', 'name', 'city']

In [26]:
venues.head()

,cn_tag,longitude,latitude,category,avg_rating,avg_price,name,city
0,囊包肉,121.490440,31.288305,新疆菜,30.0,81.0,阿凡提美食娱乐城(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),上海
1,大盘鸡,121.490440,31.288305,新疆菜,30.0,81.0,阿凡提美食娱乐城(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),上海
2,羊肉串,121.490440,31.288305,新疆菜,30.0,81.0,阿凡提美食娱乐城(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),上海
3,三文鱼,121.499664,31.239138,自助餐,30.0,296.0,东方明珠旋转餐厅(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),上海
4,法式蜗牛,121.499664,31.239138,自助餐,30.0,296.0,东方明珠旋转餐厅(这是一条测试商户数据，仅用于测试开发，开发完成后请申请正式数据...),上海
